We want to experiment with BVP and RMC for this we need green functions see period2
we will be using the green functions of $y''$
## boundary green function of $y''$

$$
\begin{align*}
G''_{b_{0}}(t) &= 0, G(b_{0})=1,G(b_{1})=0 \Rightarrow \\
G_{b_{0}}(t) &= \frac{b_{1}-t}{b_{1}-b_{0}} \\   
\end{align*}
$$ 
$$
\begin{align*}
G''_{b_{1}}(t) &= 0, G(b_{0})=0,G(b_{1})=1 \Rightarrow \\
G_{b1}(t) &= \frac{t-b_{0}}{b_{1} -b_{0}}
\end{align*}
$$


## source green function of $y''$
$$
\begin{align*}
G''_{s}(t) &= \delta(t-s), G(b_{0})=0,G(b_{1})=0 \Rightarrow \\
G_{s}(t) &=    \begin{cases}
        -\frac{(b_{1}-s)(t-b_{0})}{b_{1}-b_{0}} & \text{if } t<s\\
        -\frac{(b_{1}-t)(s-b_{0})}{b_{1}-b_{0}} & \text{if } s<t
    \end{cases} \\
G_{t}(s) &=    \begin{cases}
        -\frac{(b_{1}-t)(s-b_{0})}{b_{1}-b_{0}} & \text{if } s<t\\
        -\frac{(b_{1}-s)(t-b_{0})}{b_{1}-b_{0}} & \text{if } t<s
    \end{cases}
\end{align*} 
$$ 

## tests of the green function
We solve
$$
y''= y,y(b_{0}), y(b_{1}).
$$ 
to test these green functions are correct



In [197]:
from random import random
from math import exp

def Gb0(t,b0,b1): return (b1-t)/(b1-b0)
def Gb1(t,b0,b1): return (t-b0)/(b1-b0)
def G(t,s,b0,b1): return -(b1-s)*(t-b0)/(b1-b0) if t<s else - (b1-t)*(s-b0)/(b1-b0) 

def Y(t,y0,y1,b0,b1): 
    l = 2 # russian roulette rate
    if random()*l>1: return Gb0(t,b0,b1)*y0 + Gb1(t,b0,b1)*y1
    S = b0+random()*(b1-b0)
    return Gb0(t,b0,b1)*y0 + Gb1(t,b0,b1)*y1 + l*G(t,S,b0,b1) * Y(S,y0,y1,b0,b1)*(b1-b0)
    
def soltest(t,y0,y1,b0,b1,nsim): return sum(Y(t,y0,y1,b0,b1) for _ in range(nsim))/nsim

t,nsim = 0,10**2
print((soltest(t,exp(-0.5),exp(0.5),-0.5,0.5,nsim)-exp(t))/exp(t))

0.036561358882911676


some old text (in the comments)
<!-- Lets now do a linear boundary problem:
$$
y''= y, y(-1)=e^{-1},y(1)=e^{1}.
$$ 
with somekind of RRMC, coupled recursion and recursion on the boundary. To turn this into an integral equation we will use green functions. We will use the green function of $y''$ on $[0,1]$ because we can because we can abuse symmetries (this is a special case of walk on spheres).

We will do 2-phase coupled recursion with $y(-0.5),y(0.5)$ and $y(0)$ by integrating over $[-0.5,0.5]$ and $[-1,0],[0,1]$ but lets first test if the translation symmetry works by solving on these domains with exact boundaries. -->

Our main example will be solving 
$$
y''=y, y(-1) = e^{-1},y(1)=e .
$$
on equally spaced grid so we can test convergence stuff. But first will do a subcase of this a grid with $3$ points $[-0.5,0,0.5]$. <br> 

We will implement this with recursion in recursion. The outer recursion is a coupled recursion of $3$ points and the inner recursion is just regular RMC where we assume we know the value at adjacent points.

In [260]:
from random import random
from math import exp
import numpy as np

def Gb0(t,b0,b1): return (b1-t)/(b1-b0)
def Gb1(t,b0,b1): return (t-b0)/(b1-b0)
def G(t,s,b0,b1): return -(b1-s)*(t-b0)/(b1-b0) if t<s else - (b1-t)*(s-b0)/(b1-b0) 

def Y(t,y0,y1,b0,b1): 
    l = 2 # russian roulette rate
    if random()*l>1: return Gb0(t,b0,b1)*y0 + Gb1(t,b0,b1)*y1
    S = b0+random()*(b1-b0)
    return Gb0(t,b0,b1)*y0 + Gb1(t,b0,b1)*y1 + l*G(t,S,b0,b1) * Y(S,y0,y1,b0,b1)*(b1-b0)

def X_slow():
    l = 1.1 # russian roulette rate
    if random()*l>1: return  np.zeros(3)
    X = X_slow()
    x0 = Y(-0.5,exp(-1),X[1],-1,0)
    x1 = Y(0,X[0],X[2],-0.5,0.5)
    x2 = Y(0.5,X[1],exp(1),0,1) 
    return l*np.array([x0,x1,x2]) 

def soltest2(nsim): return sum(X_slow() for _ in range(nsim))/nsim
grid = np.array([-0.5,0,0.5])
nsim = 10**3
sol = np.exp(grid)
ss = soltest2(nsim)
print(f"soltest2= {ss}, %error{(ss-sol)/sol}")

soltest2= [0.58966353 0.99372861 1.62955266], %error[-0.0278092  -0.00627139 -0.01162635]


We want to CV the following integral equation:
$$
\begin{align*}
y(t)&=  G_{b_{0}}(t) y(b_{0}) + G_{b_{1}}(t) y(b_{1}) + \int_{b_{0}}^{b_{1}} G_{t}(s) f(s)ds \\
    &=  G_{b_{0}}(t) y(b_{0}) + G_{b_{1}}(t) y(b_{1}) + \int_{b_{0}}^{b_{1}} G_{t}(s) y(s)ds
\end{align*}
$$ 
Here a is the $0th$-order CV:
$$
G_{t}(s)y(s) \approx \frac{G_{t}(t) (y(b_{0}) +y(b_{1}))}{4}
$$ 
$1st$-order CV:
$$
y(s)= y(b_{0}) + \frac{s-b_{0}}{b_{1}-b_{0}} ( y(b_{1})-y(b_{0})).
$$ 
and the green function is piecewise linear. <br>
Ok integrating the control variates:
$$
\begin{align*}
    
\int_{b_{0}}^{b_{1}} \frac{G_{t}(t) (y(b_{0}) +y(b_{1}))}{4} ds 
&= \frac{G_{t}(t) (y(b_{0}) +y(b_{1}))(b_{1}-b_{0})}{4} \\
\int_{b_{0}}^{b_{1}} G_{t}(s) \left(y(b_{0}) + \frac{s-b_{0}}{b_{1}-b_{0}} ( y(b_{1})-y(b_{0}))  \right)ds &= \\ 
\end{align*}

$$ 




We did some chatgpt to get:<br>
implement the following function in sympy:
$$
G_{t}(s) =    \begin{cases}
        -\frac{(b_{1}-t)(s-b_{0})}{b_{1}-b_{0}} & \text{if } s<t\\
        -\frac{(b_{1}-s)(t-b_{0})}{b_{1}-b_{0}} & \text{if } t<s
    \end{cases}
$$
where $b_{0},b_{1},t,s$ are parameters

In [272]:
import sympy as sp

def G(s, t, b0, b1):
    return sp.Piecewise(
        ((b1 - t) * (s - b0) / (b1 - b0), s < t),
        ((b1 - s) * (t - b0) / (b1 - b0), t <= s)
    )

# Second derivative of Gt with respect to s
s, t, b0, b1 = sp.symbols('s t b0 b1')
Gt_second_deriv = sp.diff(G(s, t, b0, b1), s, 1)

# Print the result
print(Gt_second_deriv)


Piecewise(((b1 - t)/(-b0 + b1), s < t), (-(-b0 + t)/(-b0 + b1), True))


implement to following function in sympy:
$$
y(s)= y(b_{0}) + \frac{s-b_{0}}{b_{1}-b_{0}} ( y(b_{1})-y(b_{0})).
$$ 
where $y(b_{0}),y(b_{1}),b_{0},b_{1},t,s$ are parameters

In [277]:
import sympy as sp

def y(s, b0, b1, y_b0, y_b1):
    return y_b0 + (s - b0) / (b1 - b0) * (y_b1 - y_b0)

s, b0, b1, y_b0, y_b1 = sp.symbols('s b0 b1 y_b0 y_b1')

integral = sp.integrate(y(s, b0, b1, y_b0, y_b1), (s, b0, b1))
integral = sp.simplify(integral)
display(integral)

-b0*y_b0/2 - b0*y_b1/2 + b1*y_b0/2 + b1*y_b1/2

Given:

def G(s, t, b0, b1):
    return sp.Piecewise(
        ((b1 - t) * (s - b0) / (b1 - b0), s < t),
        ((b1 - s) * (t - b0) / (b1 - b0), t <= s)
    )
and 

def y(s, b0, b1, y_b0, y_b1):
    return y_b0 + (s - b0) / (b1 - b0) * (y_b1 - y_b0)

integrate G y in s from b0 in b1 in sympy


In [278]:
import sympy as sp

def G(s, t, b0, b1):
    return sp.Piecewise(
        ((b1 - t) * (s - b0) / (b1 - b0), s < t),
        ((b1 - s) * (t - b0) / (b1 - b0), t <= s)
    )

def y(s, b0, b1, y_b0, y_b1):
    return y_b0 + (s - b0) / (b1 - b0) * (y_b1 - y_b0)

s, t, b0, b1, y_b0, y_b1 = sp.symbols('s t b0 b1 y_b0 y_b1')

integrand = G(s, t, b0, b1) * y(s, b0, b1, y_b0, y_b1)
integral = sp.integrate(integrand, (s, b0, b1))
integral = sp.simplify(integral)
display(integral)

Piecewise(((-b0**2*b1*y_b0 - 2*b0**2*b1*y_b1 + b0**2*t*y_b0 + 2*b0**2*t*y_b1 + 2*b0*b1**2*y_b0 + b0*b1**2*y_b1 - 2*b0*b1*t*y_b0 + 2*b0*b1*t*y_b1 - 6*b0*t*y_b1*Min(b1, Max(b0, t)) + 3*b0*y_b1*Min(b1, Max(b0, t))**2 - 2*b1**2*t*y_b0 - b1**2*t*y_b1 + 6*b1*t*y_b0*Min(b1, Max(b0, t)) - 3*b1*y_b0*Min(b1, Max(b0, t))**2 - 3*t*y_b0*Min(b1, Max(b0, t))**2 + 3*t*y_b1*Min(b1, Max(b0, t))**2 + 2*y_b0*Min(b1, Max(b0, t))**3 - 2*y_b1*Min(b1, Max(b0, t))**3)/(6*(b0 - b1)), b0 < b1), ((2*b0**3*y_b0 + b0**3*y_b1 - 4*b0**2*b1*y_b0 - 2*b0**2*b1*y_b1 - 2*b0**2*t*y_b0 - b0**2*t*y_b1 + 2*b0*b1**2*y_b0 + 4*b0*b1**2*y_b1 + 4*b0*b1*t*y_b0 - 4*b0*b1*t*y_b1 + 6*b0*t*y_b1*Min(b0, Max(b1, t)) - 3*b0*y_b1*Min(b0, Max(b1, t))**2 - b1**3*y_b0 - 2*b1**3*y_b1 + b1**2*t*y_b0 + 2*b1**2*t*y_b1 - 6*b1*t*y_b0*Min(b0, Max(b1, t)) + 3*b1*y_b0*Min(b0, Max(b1, t))**2 + 3*t*y_b0*Min(b0, Max(b1, t))**2 - 3*t*y_b1*Min(b0, Max(b1, t))**2 - 2*y_b0*Min(b0, Max(b1, t))**3 + 2*y_b1*Min(b0, Max(b1, t))**3)/(6*(b0 - b1)), True))

In [302]:
import sympy as sp

def G(s, t, b0, b1):
    return sp.Piecewise(
        ((b1 - t) * (s - b0) / (b1 - b0), s < t),
        ((b1 - s) * (t - b0) / (b1 - b0), t <= s)
    )

def y(s, b0, b1, y_b0, y_b1):
    return y_b0 + (s - b0) / (b1 - b0) * (y_b1 - y_b0)

s, t, b0, b1, y_b0, y_b1 = sp.symbols('s t b0 b1 y_b0 y_b1')
sp.assumptions.assume(sp.Q.lt(b0,b1))
with sp.assuming():
    integrand = G(s, t, b0, b1) * y(s, b0, b1, y_b0, y_b1)
    integral = sp.integrate(integrand, (s, b0, b1))
    integral = sp.simplify(integral)
    display(integral)




TypeError: 'module' object is not callable

In [322]:
import sympy as sp

x, y = sp.symbols('x y')
expr = sp.Min(x, y)

assumption = sp.Q.lt(thu)

simplified_expr = sp.simplify(expr, assumptions=assumption)
print(simplified_expr)  # prints x


Min(x, y)


In [ ]:
from random import random
from math import exp

def Gb0(t,b0,b1): return (b1-t)/(b1-b0)
def Gb1(t,b0,b1): return (t-b0)/(b1-b0)
def G(t,s,b0,b1): return -(b1-s)*(t-b0)/(b1-b0) if t<s else - (b1-t)*(s-b0)/(b1-b0) 

def Y(t,y0,y1,b0,b1): 
    l = 2 # russian roulette rate
    if random()*l>1: return Gb0(t,b0,b1)*y0 + Gb1(t,b0,b1)*y1
    S = b0+random()*(b1-b0)
    return Gb0(t,b0,b1)*y0 + Gb1(t,b0,b1)*y1 + l*G(t,S,b0,b1) * Y(S,y0,y1,b0,b1)*(b1-b0)
    
def soltest(t,y0,y1,b0,b1,nsim): return sum(Y(t,y0,y1,b0,b1) for _ in range(nsim))/nsim

t,nsim = 0,10**2
print((soltest(t,exp(-0.5),exp(0.5),-0.5,0.5,nsim)-exp(t))/exp(t))